# 17기 KNN 정규세션 과제

## KNN 구현해보기
### 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.
### 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.
### 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

**데이터:** [blackfriday | Kaggle](https://www.kaggle.com/llopesolivei/blackfriday)

---

## 0. 데이터 불러오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Tobigs/week4/Tobigs18기_Week4_KNN&Clustering/과제/blackfriday.csv", index_col = 0)
df.head()

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1001088,P00046042,F,0-17,10,A,3,0,5,17.0,NaN,2010
1,1004493,P00347742,F,0-17,10,A,1,0,7,NaN,NaN,4483
2,1005302,P00048942,F,0-17,10,A,1,0,1,4.0,NaN,7696
3,1001348,P00145242,F,0-17,10,A,3,0,2,4.0,NaN,16429
4,1001348,P00106742,F,0-17,10,A,3,0,3,5.0,NaN,5780


## 1. Preprocssing / EDA
지금까지 배운 내용을 토대로 해당 데이터에 대해 자유롭게 전처리와 EDA를 진행해주세요.

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   User_ID                     4998 non-null   int64  
 1   Product_ID                  4998 non-null   object 
 2   Gender                      4998 non-null   object 
 3   Age                         4998 non-null   object 
 4   Occupation                  4998 non-null   int64  
 5   City_Category               4998 non-null   object 
 6   Stay_In_Current_City_Years  4998 non-null   object 
 7   Marital_Status              4998 non-null   int64  
 8   Product_Category_1          4998 non-null   int64  
 9   Product_Category_2          3465 non-null   float64
 10  Product_Category_3          1544 non-null   float64
 11  Purchase                    4998 non-null   int64  
dtypes: float64(2), int64(5), object(5)
memory usage: 507.6+ KB


결측치가 있는 변수가 있다...

In [ ]:
df.describe()

,User_ID,Occupation,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
count,4.998000e+03,4998.000000,4998.000000,4998.000000,3465.000000,1544.000000,4998.000000
mean,1.003015e+06,8.036815,0.421369,5.320128,9.773737,12.766839,9234.655462
std,1.709910e+03,6.442697,0.493828,3.912281,5.101867,4.023591,4999.225081
min,1.000003e+06,0.000000,0.000000,1.000000,2.000000,3.000000,13.000000
25%,1.001560e+06,2.000000,0.000000,1.000000,5.000000,9.000000,5497.750000
50%,1.003040e+06,7.000000,0.000000,5.000000,9.000000,14.000000,8049.000000
75%,1.004417e+06,14.000000,1.000000,8.000000,15.000000,16.000000,12039.000000
max,1.006037e+06,20.000000,1.000000,20.000000,18.000000,18.000000,23913.000000


In [ ]:
df['Product_Category_1'].unique()

array([ 5,  7,  1,  2,  3,  8, 16, 14, 10, 19, 11, 12,  4, 17, 15,  6, 13,
       20, 18,  9])

In [ ]:
df['Product_Category_2'].unique()

array([17., nan,  4.,  5., 15.,  8., 12., 11.,  2., 14.,  3., 18., 16.,
        6., 13., 10.,  9.,  7.])

In [ ]:
df['Product_Category_3'].unique()

array([nan,  9.,  5., 15.,  8., 14., 16., 17.,  4., 18., 12., 13.,  6.,
       10., 11.,  3.])

In [ ]:
df['Age'].unique() #나이 순으로 라벨링 해도 될것 같다.

array(['0-17', '18-25', '26-35', '36-45', '46-50', '51-55', '55+'],
      dtype=object)

In [ ]:
df['City_Category'].unique() #unique값이 그리 많지 않으니....one hot encoding하기

array(['A', 'B', 'C'], dtype=object)

In [ ]:
df['Stay_In_Current_City_Years'].unique() #이것도 거주 연도 순으로 라벨링 해도 될것 같다.

array(['3', '1', '2', '4+', '0'], dtype=object)

저 product category를 어떻게 해야할지 잘 모르겠다....<br>
product category3 같은 경우는 결측치가 채워진 값보다 더 많다...그냥 지우는게 나을것 같다. <br>
그리고 product category2 같은 경우는 kaggle홈페이지 들어가보니깐 빈도수가 제일 많은 관측치로 넣었길래 그렇게 하기로 했다.

In [ ]:
#빈도수 제일 많은 관측치로 채우기
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')
df[['Product_Category_2','Product_Category_3']] = imputer.fit_transform(df[['Product_Category_2','Product_Category_3']])

In [ ]:
#product_category_3 지우기
df=df.drop(labels='Product_Category_3',axis=1)

In [ ]:
#도시 카테고리 원핫인코딩
df=pd.get_dummies(df, columns = ['City_Category'])
df

,User_ID,Product_ID,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase,City_Category_A,City_Category_B,City_Category_C
0,1001088,P00046042,F,0-17,10,3,0,5,17.0,2010,1,0,0
1,1004493,P00347742,F,0-17,10,1,0,7,8.0,4483,1,0,0
2,1005302,P00048942,F,0-17,10,1,0,1,4.0,7696,1,0,0
3,1001348,P00145242,F,0-17,10,3,0,2,4.0,16429,1,0,0
4,1001348,P00106742,F,0-17,10,3,0,3,5.0,5780,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,1000837,P00147942,M,55+,7,1,1,1,16.0,11701,0,0,1
4994,1000293,P00358242,M,55+,1,1,1,8,8.0,2045,0,0,1
4995,1001908,P00173342,M,55+,13,3,1,1,6.0,4505,0,0,1
4996,1005517,P00336742,M,55+,0,2,1,12,8.0,1413,0,0,1


In [ ]:
label={'0-17':1, '18-25':2, '26-35':3, '36-45':4, '46-50':5, '51-55':6, '55+':7}
df['Age']=df['Age'].map(label)
df

,User_ID,Product_ID,Gender,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase,City_Category_A,City_Category_B,City_Category_C
0,1001088,P00046042,F,1,10,3,0,5,17.0,2010,1,0,0
1,1004493,P00347742,F,1,10,1,0,7,8.0,4483,1,0,0
2,1005302,P00048942,F,1,10,1,0,1,4.0,7696,1,0,0
3,1001348,P00145242,F,1,10,3,0,2,4.0,16429,1,0,0
4,1001348,P00106742,F,1,10,3,0,3,5.0,5780,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,1000837,P00147942,M,7,7,1,1,1,16.0,11701,0,0,1
4994,1000293,P00358242,M,7,1,1,1,8,8.0,2045,0,0,1
4995,1001908,P00173342,M,7,13,3,1,1,6.0,4505,0,0,1
4996,1005517,P00336742,M,7,0,2,1,12,8.0,1413,0,0,1


In [ ]:
#성별 원핫인코딩
df=pd.get_dummies(df, columns = ['Gender'])
df

,User_ID,Product_ID,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase,City_Category_A,City_Category_B,City_Category_C,Gender_F,Gender_M
0,1001088,P00046042,1,10,3,0,5,17.0,2010,1,0,0,1,0
1,1004493,P00347742,1,10,1,0,7,8.0,4483,1,0,0,1,0
2,1005302,P00048942,1,10,1,0,1,4.0,7696,1,0,0,1,0
3,1001348,P00145242,1,10,3,0,2,4.0,16429,1,0,0,1,0
4,1001348,P00106742,1,10,3,0,3,5.0,5780,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,1000837,P00147942,7,7,1,1,1,16.0,11701,0,0,1,0,1
4994,1000293,P00358242,7,1,1,1,8,8.0,2045,0,0,1,0,1
4995,1001908,P00173342,7,13,3,1,1,6.0,4505,0,0,1,0,1
4996,1005517,P00336742,7,0,2,1,12,8.0,1413,0,0,1,0,1


In [ ]:
labels={'3':3, '1':1, '2':2, '4+':4, '0':0}
df['Stay_In_Current_City_Years']=df['Stay_In_Current_City_Years'].map(labels)
df

,User_ID,Product_ID,Age,Occupation,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Purchase,City_Category_A,City_Category_B,City_Category_C,Gender_F,Gender_M
0,1001088,P00046042,1,10,3,0,5,17.0,2010,1,0,0,1,0
1,1004493,P00347742,1,10,1,0,7,8.0,4483,1,0,0,1,0
2,1005302,P00048942,1,10,1,0,1,4.0,7696,1,0,0,1,0
3,1001348,P00145242,1,10,3,0,2,4.0,16429,1,0,0,1,0
4,1001348,P00106742,1,10,3,0,3,5.0,5780,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4993,1000837,P00147942,7,7,1,1,1,16.0,11701,0,0,1,0,1
4994,1000293,P00358242,7,1,1,1,8,8.0,2045,0,0,1,0,1
4995,1001908,P00173342,7,13,3,1,1,6.0,4505,0,0,1,0,1
4996,1005517,P00336742,7,0,2,1,12,8.0,1413,0,0,1,0,1


In [ ]:
#아이디 제거
df=df.drop(['User_ID','Product_ID'],axis=1)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4998 entries, 0 to 4997
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         4998 non-null   int64  
 1   Occupation                  4998 non-null   int64  
 2   Stay_In_Current_City_Years  4998 non-null   int64  
 3   Marital_Status              4998 non-null   int64  
 4   Product_Category_1          4998 non-null   int64  
 5   Product_Category_2          4998 non-null   float64
 6   Purchase                    4998 non-null   int64  
 7   City_Category_A             4998 non-null   uint8  
 8   City_Category_B             4998 non-null   uint8  
 9   City_Category_C             4998 non-null   uint8  
 10  Gender_F                    4998 non-null   uint8  
 11  Gender_M                    4998 non-null   uint8  
dtypes: float64(1), int64(6), uint8(5)
memory usage: 336.8 KB


## 2. KNN 구현 & 파라미터 튜닝
수업 내용 및 실습 자료를 참고하여 KNN을 구현하고 파라미터 튜닝을 하며 결과를 비교해주세요.

결과를 비교하려면....Y값이 있어야 한다고 생각하는데...<br>
Y값이 뭔지는 모르겠지만 일단 kaggle 페이지에 purchase변수가 없기 때문에<br> 해당 변수를 y값으로 둔다.

In [ ]:
y=df['Purchase']
x=df.drop('Purchase',axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, shuffle=True, random_state=34)

### KNN

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
regressor1 = KNeighborsRegressor(n_neighbors = 3, weights = "uniform")
regressor1.fit(x_train, y_train)
guesses1 = regressor1.predict(x_valid)

In [ ]:
print(regressor1.score(x_train, y_train))
print(regressor1.score(x_valid, guesses1)) #????!??!?!?!?!?!

0.6794812043056205
1.0


참고 사이트:https://rebro.kr/184

### weight KNN

In [ ]:
regressor2 = KNeighborsRegressor(n_neighbors = 3, weights = "distance")
regressor2.fit(x_train, y_train)
guesses2 = regressor2.predict(x_valid)

In [ ]:
print(regressor2.score(x_train, y_train))
print(regressor2.score(x_valid, guesses2))

0.9642264750996573
1.0


### grid search

In [ ]:
# grid search
from sklearn.model_selection import GridSearchCV

In [ ]:
grid_params = {
    'n_neighbors' : list(range(1,20)),
    'weights' : ["uniform", "distance"],
    'metric' : ['euclidean', 'manhattan', 'minkowski']
}

In [ ]:
knn=KNeighborsRegressor()

In [ ]:
gs = GridSearchCV(knn, grid_params, cv=10)
gs.fit(x_train, y_train)
print("Best Parameters : ", gs.best_params_)
print("Best Score : ", gs.best_score_)
print("Best Test Score : ", gs.score(x_valid, y_valid))

Best Parameters :  {'metric': 'manhattan', 'n_neighbors': 11, 'weights': 'uniform'}
Best Score :  0.37369104664265523
Best Test Score :  0.3651291981805864


참고하면 좋을 사이트: https://tobigs.gitbook.io/tobigs/data-analysis/python-knn

## 3. Evaluation
결과에 대한 평가를 진행하고, 나름의 해석을 달아주세요.

In [ ]:
# evaluation
from sklearn.model_selection import cross_val_score
from sklearn.metrics import *